## 텍스트 분류(Text Classification) 실습
- BERT 다국어 모델과 BERT 한국어 모델 비교 평가
- BERT 다국어 모델을 이용하여 리뷰의 감성을 예측하는 모델 생성, 
- 학습용 데이터를 이용하여 학습, 평가용 데이터를 이용하여 모델 평가
- BERT 한국어 모델을 이용하여 리뷰의 감성을 예측하는 모델 생성,
- 학습용 데이터를 이용하여 학습, 평가용 데이터를 이용하여 모델 평가

In [ ]:
'''
실습 데이터 : 네이버 영화 리뷰 감성분석 데이터
네이버 영화의 댓글을 모아 구성된 한국어 텍스트 -> 컬럼 : id, document(리뷰), label(부정, 긍정)
학습용 데이터 : 150,000개, 평가용 데이터 50,000개

'''

In [ ]:
#-함수 및 메서드 정리
#사전 학습된 토크나이저 객체 생성 / 사전 학습된 모델객체 생성
# from_pretrained() 메소드 사용
#
#BertTokenizer.from_pretrained(model_name, cache_dir, do_lower_case=False)
#TFBertModel.from_pretrained(model_name, cache_dir)

'''

encode_plus() 매서드
1) encode()의 기능 확장
2) 텍스트 데이터를 BERT에 필요한 입력 형태로 변환
3) 문장을 최대 길이에 맞게 패딩
4) 결과값을 딕셔너리로 출력
1. input_ids : 단어 사전을 참조하여 문장을 단어의 인덱스로 변환, BERT에 필요한 입력 형태 제공
2. attention_mask : 패딩이 아닌 단어와 패딩인 부분을 구분(패딩이 아닌 단어:1, 패딩:0)
3. token_type_ids : 입력된 두 문장을 구분(첫번째 문장:0, 두번째 문장:1)

'''

In [ ]:
''' 

매개변수

1. text : 입력 문장
2. add_special_tokens: 문장의 시작점에 '[CLS]' 토큰, 각 문장의 마지막에 '[SEP]' 토큰 생성
3. max_length : 지정된 최대 길이에 따라 입력 문장의 길이를 맞춤, 자르고 붙이기
4. pad_to_max_length : max_length 길이에 미치지 못하는 문장에 패딩 적용
5. return_attention_mask : 어텐션 마스크를 생성

예시
		encoded_dict = tokenizer.encode_plus(
			text = sent,
			add_special_tokens = True,
			max_length = MAX_LEN,
			pad_to_max_length = True,
			return_attention_mask = True
)

문장이 정수로 바뀌는 거 외에 encodeplus하면 3가지 다 만들어주면 그 결과값을 편리하게 쓰게 만들어준다.
encode.plus함수를 써서 BERT 모델에 입력하는 형태로 텍스트 데이터를 변환할것이다. 이게 핵심적인 개념.

subword 방식이기 때문에 불용어는 빼지 않는다. 불필요한 ... 이런거 빼주고 이모티콘도 다 넣는다.
공백은 남겨놓고 ㅎㅎ.. 조사, 어미 그대로 집어넣을 것.

알아서 bert토크나이저가 subword tokenizer방식으로 알아서 되게 만들 것.

문장부호, 기호만 제거할 것. 그게 전처리. 이전의 전처리방식과는 다르다.


'''

## BERT 모델을 이용한 텍스트 감성 분석

### BERT 다국어 모델 / 한국어 모델 Tokenizer 비교 테스트

In [ ]:
### 필요한 라이브러리 임폴트
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import tensorflow as tf

from transformers import pipeline, set_seed, BertTokenizer, TFBertForSequenceClassification

## Sequence 단어 토큰이 나열되는 것. text를 의미함.
## text분류의 Bert모델이다 라고 생각하면 됨.
## 앞에 TF가 없으면 PyTorch로 만들어지는 것.
## 앞에 TF가 있으면 TensorFlowㄹ 만들어지는 것.

